In [2]:
import tensorflow as tf
import numpy as np
import os, glob
tfk = tf.keras

# ML
import sys
from cymetric.models.tfhelper import prepare_tf_basis, train_model

# training
from cymetric.models.tfmodels import PhiFSModel

sys.path.append("../..")
from utils import eijk

from models import ProjectiveSpace

In [3]:
dirname = 'QuinticData'
data = np.load(os.path.join(dirname, 'dataset.npz'))
BASIS = np.load(os.path.join(dirname, 'basis.pickle'), allow_pickle=True)

In [4]:
from cymetric.models.tfhelper import prepare_tf_basis, train_model

In [5]:
BASIS = prepare_tf_basis(BASIS)

In [6]:
kappa = np.real(BASIS['KAPPA'].numpy());
kappa


0.017621975

In [7]:
from cymetric.models.callbacks import RicciCallback, SigmaCallback, VolkCallback, KaehlerCallback, TransitionCallback
from cymetric.models.tfmodels import MultFSModel
from cymetric.models.metrics import SigmaLoss, KaehlerLoss, TransitionLoss, VolkLoss, RicciLoss, TotalLoss

In [8]:
rcb = RicciCallback((data['X_val'], data['y_val']), data['val_pullbacks'])
scb = SigmaCallback((data['X_val'], data['y_val']))
volkcb = VolkCallback((data['X_val'], data['y_val']))
kcb = KaehlerCallback((data['X_val'], data['y_val']))
tcb = TransitionCallback((data['X_val'], data['y_val']))
cb_list = [rcb, scb, kcb, tcb, volkcb]

In [9]:
nlayer = 3
nHidden = 64
act = 'gelu'
nEpochs = 5
bSizes = [64, 5000]
alpha = [1., 1., 1., 1., 1.]
nfold = 3
n_in = 2*5
n_out = 1

In [10]:
nn = tf.keras.Sequential()
nn.add(tfk.Input(shape=(n_in)))
for i in range(nlayer):
    nn.add(tfk.layers.Dense(nHidden, activation=act))
nn.add(tfk.layers.Dense(n_out, use_bias=False))

In [11]:
fmodel = PhiFSModel(nn, BASIS, alpha=alpha)

In [11]:
cmetrics = [TotalLoss(), SigmaLoss(), KaehlerLoss(), TransitionLoss(), VolkLoss(), RicciLoss()]
opt = tfk.optimizers.Adam()

In [12]:
fmodel, training_history = train_model(fmodel, data, optimizer=opt, epochs=nEpochs, batch_sizes=[64, 5000], 
                                       verbose=1, custom_metrics=cmetrics, callbacks=cb_list)


Epoch  1/5
80/80 [==============================] - 36s 392ms/step - loss: 0.2412 - sigma_loss: 0.1527 - kaehler_loss: 0.0000e+00 - transition_loss: 0.0000e+00 - volk_loss: 0.0885 - ricci_loss: 0.0000e+00
 - Ricci measure val:      1.2433
 - Sigma measure val:      0.1428
 - Kaehler measure val:    9.9124e-16
 - Transition measure val: 0.0027
 - Volk val:               5.1413

Epoch  2/5
80/80 [==============================] - 36s 412ms/step - loss: 0.1010 - sigma_loss: 0.0511 - kaehler_loss: 0.0000e+00 - transition_loss: 0.0000e+00 - volk_loss: 0.0499 - ricci_loss: 0.0000e+00
 - Ricci measure val:      0.2173
 - Sigma measure val:      0.0427
 - Kaehler measure val:    1.4002e-15
 - Transition measure val: 0.0019
 - Volk val:               5.1116

Epoch  3/5
80/80 [==============================] - 36s 407ms/step - loss: 0.1035 - sigma_loss: 0.0361 - kaehler_loss: 0.0000e+00 - transition_loss: 0.0000e+00 - volk_loss: 0.0674 - ricci_loss: 0.0000e+00
 - Ricci measure val:      0.1176


In [13]:
x_vars = tf.cast(tf.constant(data["X_train"]), tf.float32)
weights, omegas = data['y_train'][:,-2], data['y_train'][:,-1]

print(np.mean(weights * np.linalg.det(fmodel(x_vars))/ omegas))

(3.6117253955428716-1.9305438481768693e-09j)


In [14]:
fmodel(x_vars[0:1])

<tf.Tensor: shape=(1, 3, 3), dtype=complex64, numpy=
array([[[ 0.09253744-7.5669959e-10j,  0.00856687-1.9730455e-03j,
         -0.00577878-2.5402289e-02j],
        [ 0.00856686+1.9730462e-03j,  0.07905391+1.5870683e-10j,
          0.00865713+1.0466812e-02j],
        [-0.00577878+2.5402287e-02j,  0.00865713-1.0466811e-02j,
          0.07936295+6.9849193e-10j]]], dtype=complex64)>

In [18]:
np.shape(x_vars)

TensorShape([400000, 10])

In [20]:
x_vars[0:1]

<tf.Tensor: shape=(1, 10), dtype=float32, numpy=
array([[ 5.9635967e-01, -2.0448701e-01,  8.1529528e-01,  1.0000000e+00,
         7.5759637e-01, -4.6444046e-01,  3.4482372e-01,  5.0856006e-01,
        -3.4694470e-18, -2.3732327e-01]], dtype=float32)>

In [19]:
def ricci_scalar_fn(model, points, pullbacks=None, verbose=0, rdet=True):
    r"""Computes the Ricci scalar for a kaehler metric.

    .. math::
        R = g^{ij} \partial_i \bar{\partial}_j \log \det g

    Args:
        model (tfk.model): Any (sub-)class of FSModel.
        points (tensor[(n_p,2*ncoord), tf.float32]): NN input
        pullbacks (tensor[(n_p,nfold,ncoord), tf.complex64]): Pullback tensor. Defaults to None. Then gets computed.
        verbose (int, optional): if > 0 prints some intermediate infos. Defaults to 0.
        rdet (bool, optional): if True also returns det. Defaults to True.
            This is a bit hacky, because the output signature changes
            but avoids recomputing the determinant after batching.

    Returns:
        tf.float32(tensor[(n_p,), tf.float32]): Ricci scalar
    """
    ncoords = model.ncoords
    x_vars = points
    if pullbacks is None:
        pullbacks = model.pullbacks(points)
    # take derivatives
    with tf.GradientTape(persistent=True) as tape1:
        tape1.watch(x_vars)
        with tf.GradientTape(persistent=True) as tape2:
            tape2.watch(x_vars)
            prediction = model(x_vars)
            det = tf.math.real(tf.linalg.det(prediction)) * 1.  # factorial / (2**nfold)
            log = tf.math.log(det)
        di_dg = tape2.gradient(log, x_vars)
    didj_dg = tf.cast(tape1.batch_jacobian(di_dg, x_vars), dtype=tf.complex64)
    # add derivatives together to complex tensor
    ricci_ij = didj_dg[:, 0:ncoords, 0:ncoords]
    ricci_ij += 1j * didj_dg[:, 0:ncoords, ncoords:]
    ricci_ij -= 1j * didj_dg[:, ncoords:, 0:ncoords]
    ricci_ij += didj_dg[:, ncoords:, ncoords:]
    ricci_ij *= 0.25
    pred_inv = tf.linalg.inv(prediction)
    ricci_scalar = tf.einsum('xba,xai,xij,xbj->x', pred_inv, pullbacks,
                             ricci_ij, tf.math.conj(pullbacks))
    ricci_scalar = tf.math.real(ricci_scalar)
    if verbose > 0:
        tf.print(' - Avg ricci scalar is',
                 tf.math.reduce_mean(ricci_scalar), output_stream=sys.stdout)
        if verbose > 1:
            tf.print(' - Max ricci scalar is',
                     tf.reduce_max(ricci_scalar), output_stream=sys.stdout)
            tf.print(' - Min ricci scalar is',
                     tf.reduce_min(ricci_scalar), output_stream=sys.stdout)
    if rdet:
        return ricci_scalar, det
    else:
        return ricci_scalar

In [25]:
riem = None
getNum = lambda x: int(x.split('/')[1].split('.')[0][4:])
files = sorted(glob.glob("riemvalues/*.npy"), key = getNum)
for riemfile in files:
    if riem is None:
        riem = np.load(riemfile)
    else:
        riem = np.append(riem, np.load(riemfile), axis=0)
riem = np.asarray(riem)

In [42]:
ltot=10
for k in range(ltot):
    print(ricci_scalar_fn(fmodel,x_vars[k:k+1]))
    

(<tf.Tensor: shape=(1,), dtype=float32, numpy=array([8.097036], dtype=float32)>, <tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.00049893], dtype=float32)>)
(<tf.Tensor: shape=(1,), dtype=float32, numpy=array([-21.19727], dtype=float32)>, <tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.00095881], dtype=float32)>)
(<tf.Tensor: shape=(1,), dtype=float32, numpy=array([-13.258491], dtype=float32)>, <tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.00165956], dtype=float32)>)
(<tf.Tensor: shape=(1,), dtype=float32, numpy=array([-7.197782], dtype=float32)>, <tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.00073135], dtype=float32)>)
(<tf.Tensor: shape=(1,), dtype=float32, numpy=array([-27.086912], dtype=float32)>, <tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.00093361], dtype=float32)>)
(<tf.Tensor: shape=(1,), dtype=float32, numpy=array([-18.343073], dtype=float32)>, <tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.00115545], dtype=float32)>)
(<tf.Tensor: s

In [35]:
ricurv[1]

IndexError: too many indices for array: array is 0-dimensional, but 1 were indexed

In [26]:
c3 = ProjectiveSpace.getC3(tf.cast(riem, tf.complex128))
c3_top = ProjectiveSpace.getTopNumber(c3)

norm_factor = (-2*1j)**3 / np.math.factorial(3)
print(f"Int_X c3 = {np.real(norm_factor * np.mean(weights * c3_top / omegas))}")

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type NoneType).